# Libraries

In [2]:
%run "../main_global.ipynb"

Connection with MySQL database is ready!


In [5]:
import os
import glob
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

# Data
from sklearn.model_selection import train_test_split

# DL Model
from keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense
from keras.layers.convolutional import MaxPooling1D

# Optimizer
from tensorflow.keras.optimizers import Adam

# Plotting
import matplotlib.pyplot as plt

# Early stopping
from tensorflow.keras.callbacks import EarlyStopping

# Checkpoint
from keras.callbacks import ModelCheckpoint

[]


In [6]:
from numpy import arange, einsum

# UDF

In [7]:
class mvi_dl_data(object):
    """
    Sequential processing of time series for the MVI_DL algorithm
    """
    def __init__(self, sqlq, test_size = 0.2, dt_col = "datetime"):
        
        self.sqlq = sqlq
        self.dt_col = dt_col 
        self.test_size = test_size
        
    def raw_df(self):
        df = qdata(self.sqlq)
        
        # Case-Specific dropping of a monitoring station
        df = df.drop("NE3", axis = 1)
        
        return df
    
    def indexed_df(self):
        """
        Adding datetime as row index 
        """
        # New row index
        df = self.raw_df().set_index(self.dt_col)
        df.sort_index(inplace=True)
        
        return df
    
    def train_test_subtest_df(self):
        df = self.indexed_df()
        X_train, X_test = train_test_split(df, test_size=self.test_size, shuffle = False)
        _, X_subtest = train_test_split(X_test, test_size=self.test_size, shuffle = True)
        
        X_subtest.sort_index(inplace=True)
        
        # Delete values in X_subtest
        X_test.loc[X_subtest.index, :] = 0
        
        return X_train, X_test, X_subtest

\begin{equation}
\theta^{(l)} = f(f(f(f(g(\mathbf{X}^{(l-1)}))))) \label{dl_eq}
\end{equation}

Where:
\begin{itemize}
    \item $f$: Fully Connected layer with LeakyReLu non-linearity.
    \item $g$: Fully Connected layer with Flatten layer. 
\end{itemize}

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

In [18]:
from tensorflow.nn import leaky_relu

In [21]:
n_units= 10

def my_leaky_relu(x):
    return leaky_relu(x, alpha=0.01)

output = Dense(input, n_units, activation=my_leaky_relu)

TypeError: __init__() got multiple values for argument 'activation'

In [ ]:
tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999
    name="Adam",
    **kwargs
)

In [ ]:

def create_model(activation): 
    model = keras.models.Sequential()
    
    
    model.add(Bidirectional(LSTM(num_neurons, activation= 'relu' ), input_shape=(n_steps, n_features)))
    
    # Output layer with one neuron to a model instance (to return one output)
    model.add(Dense(1, kernel_initializer='normal',activation='linear'))
    
    
    model.add(Dense(input_shape=[28, 28]))
    model.add(Dense())
    model.add(Dense())
    model.add(Dense())
    model.add(Dense())
    model.add(Dense())
    model.add(Flatten())
    
    model.add(Dense(300, activation=activation))
    for layer in range(49):
        model.add(Dense(100, activation=activation))
    model.add(Dense(10, activation="softmax"))
    
    
    model.compile(
        optimizer='sgd', 
        loss='sparse_categorical_crossentropy', 
        metrics=['accuracy']
    )
    return model

In [9]:
class mvi_dl_main():
    """
    This class contains the sequential processing of data for the MVI-DL algorithm.

    Input:
    data: Receives a pandas dataframe to convert into numpy array
    """
    def __init__(self, sqlq):
        main_data = mvi_dl_data(sqlq)
        data = main_data.indexed_df()
        
        # Convert pandas object into numpy array
        self.X = data.to_numpy()
        
        # Mask with true values where there are values
        self.m = ~(self.X == 0)*1
        
        # Time horizon vector
        halfT = self.X.shape[0] // 2
        self.v = arange(-halfT, halfT -1, 1)
        
    def decomposition_bias(self):
        X = self.X
        m = self.m

        bias_X = X.flatten()
        bias_m = m.flatten()

        # Vector inner products: $\Sigma x_i m_i$
        bias_num = einsum('i,i', bias_X, bias_m)

        # $\Sigma m_i$
        bias_den = bias_m.sum()

        # Bias
        bias = bias_num / bias_den
        
        # X^{(bias)} \equiv X - X^{(1)}
        # h^{(1)}
        return X - bias, bias

# UDF

# Main

In [10]:
sqlq = "Select * from sima_pm25"

In [11]:
main = mvi_dl_main(sqlq)
X1, h1 = main.decomposition_bias()

In [12]:
h1

21.206843669220145

In [14]:
1e-5 + 1

1.00001